In [8]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from keras.layers import Activation, Flatten, Bidirectional
from keras.layers import Conv1D,MaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.metrics import roc_auc_score,confusion_matrix, accuracy_score, make_scorer, f1_score,precision_score,recall_score, plot_confusion_matrix
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('rslp')
nltk.download('stopwords')

from argparse import Namespace

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\cfpc2\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cfpc2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
args = Namespace(
    train_split = 0.7,
    random_state = 42,
    vocab_size = 10000,
    embedding_dim = 16,
    max_length = 120,
    batch_size=128,
    num_epochs=5,
    early_stopping_criteria=2,
    dropout_p=0.1,
    model_storage="model_storage/lstm",
)

In [3]:
dataset = pd.read_csv("datasets/reviews.csv")
X = dataset["review_comment_message"].copy()
y = dataset["review_score"].copy()
y = np.array(y)

In [4]:
for i in range(0,len(y)):
    if y[i] == -1:
        y[i] = 2
print(y)

[1 1 0 ... 0 1 2]


In [5]:
y_dummy = np_utils.to_categorical(y)
print(y_dummy)
print(y)

[[0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[1 1 0 ... 0 1 2]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y_dummy, test_size=0.3, random_state = 199)

In [14]:
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words = args.vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

def preprocess(training_sentences, testing_sentences, max_length, vocab_size, trunc_type='post', oov_tok = "<OOV>"):
    """
    Args
        training_sentences
        training_labels
        testing_sentences
        testing_labels
    Return
        training_sentences
        training_labels
        testing_sentences
        testing_labels 
    """

    stopword = stopwords.words("portuguese")
    stem = RSLPStemmer()
    vectorizer = CountVectorizer()

    def clear(review):
        review = review.lower()
        # remove pula de linha 
        review = re.sub('\n', ' ', review)        
        review = re.sub('\r', ' ', review)

        # remove numero 
        review = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' #numero ', review)

        # remove caracters especiais 
        review = re.sub(r'R\$', ' ', review)
        review = re.sub(r'\W', ' ', review)
        review = re.sub(r'\s+', ' ', review)

        # remove links 
        urls = re.findall('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', review)
        if len(urls) > 0:
            for url in urls:
                for link in url:
                    review = review.replace(link, '')
            review = review.replace(':', '')
            review = review.replace('/', '')
        return review

    training_sentences = training_sentences.apply(lambda review: clear(review))
    testing_sentences = testing_sentences.apply(lambda review: clear(review))
    training_sentences = training_sentences.apply(lambda words_review: [word for word in words_review if word not in stopword])
    testing_sentences = testing_sentences.apply(lambda words_review: [word for word in words_review if word not in stopword])
    # training_sentences = training_sentences.apply(lambda review: word_tokenize(review))
    # testing_sentences = testing_sentences.apply(lambda review: word_tokenize(review))
    training_sentences = training_sentences.apply(lambda words_review: [stem.stem(word) for word in words_review ])
    testing_sentences = testing_sentences.apply(lambda words_review: [stem.stem(word) for word in words_review ])
    training_sentences = training_sentences.apply(lambda words_review: " ".join(words_review))
    testing_sentences = testing_sentences.apply(lambda words_review: " ".join(words_review))
    training_sentences = tokenizer.texts_to_sequences(training_sentences)
    testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

    training_padded = pad_sequences(training_sentences,maxlen=max_length, truncating=trunc_type)
    testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

    # training_padded = vectorizer.fit_transform(training_sentences)
    # testing_padded = vectorizer.fit_transform(testing_sequences)

    return training_padded, testing_padded

In [15]:
X_train_new, X_test_new = preprocess(X_train, X_test, args.max_length, args.vocab_size)

In [16]:
def create_model():
    input = tf.keras.Input(shape=(args.max_length))
    x = tf.keras.layers.Embedding(args.vocab_size, args.embedding_dim, input_length=args.max_length)(input)

    x = tf.keras.layers.LSTM(16, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(16)(x)

    x = tf.keras.layers.Dropout(0.6)(x)

    x = tf.keras.layers.Dense(32, activation='relu')(x)

    output = tf.keras.layers.Dense(3, activation='softmax')(x)

    return tf.keras.Model(input, output)

In [17]:
earlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=args.early_stopping_criteria)

model = create_model()
model.compile(
  loss = tf.keras.losses.CategoricalCrossentropy(),
  optimizer= tf.keras.optimizers.Adam(
    learning_rate=0.0001),
  metrics=['accuracy']
)

history = model.fit(
    X_train_new,
    np.array(y_train), 
    validation_data=(X_test_new, np.array(y_test)),
    epochs=15,
    batch_size=args.batch_size,
    callbacks= [earlyStoppingCallback],
)

Epoch 1/15
224/224 [==============================] - 42s 174ms/step - loss: 1.0582 - accuracy: 0.4973 - val_loss: 1.0116 - val_accuracy: 0.5004
Epoch 2/15
224/224 [==============================] - 54s 241ms/step - loss: 1.0041 - accuracy: 0.5187 - val_loss: 0.9732 - val_accuracy: 0.5687
Epoch 3/15
224/224 [==============================] - 54s 243ms/step - loss: 0.9746 - accuracy: 0.5768 - val_loss: 0.9547 - val_accuracy: 0.5747
Epoch 4/15
224/224 [==============================] - 55s 243ms/step - loss: 0.9512 - accuracy: 0.5868 - val_loss: 0.9313 - val_accuracy: 0.5886
Epoch 5/15
224/224 [==============================] - 54s 243ms/step - loss: 0.9262 - accuracy: 0.6004 - val_loss: 0.9028 - val_accuracy: 0.6013
Epoch 6/15
224/224 [==============================] - 55s 244ms/step - loss: 0.9111 - accuracy: 0.6073 - val_loss: 0.8939 - val_accuracy: 0.6031
Epoch 7/15
224/224 [==============================] - 48s 215ms/step - loss: 0.9059 - accuracy: 0.6095 - val_loss: 0.8960 - val_ac

In [18]:
model.evaluate(X_test_new, np.array(y_test))

384/384 [==============================] - 7s 18ms/step - loss: 0.8810 - accuracy: 0.6071


[0.8809857368469238, 0.6071107983589172]